In [158]:
import requests
import pandas as pd

In [159]:
import simplejson as json

In [160]:
def dateFormat(date):
    return date[:25]
weather = pd.DataFrame()
df = pd.DataFrame(data['properties']['maxTemperature']['values'])
df['date'] = df['validTime'].map(dateFormat)
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True).dt.date

In [161]:
TAVG = df[['value','date']].groupby('date').max()
TAVG

,value
date,
2020-08-01,39.444444
2020-08-02,39.444444
2020-08-03,38.888889
2020-08-04,38.333333
2020-08-05,36.666667
2020-08-06,36.666667
2020-08-07,37.222222
2020-08-08,38.333333


In [162]:
from retrying import retry

def avgTemp(df):
    df['date'] = df['validTime'].map(dateFormat)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True).dt.date
    TAVG = df.groupby('date').mean()
    TAVG = TAVG*1.8+32
    return TAVG.squeeze()
def maxTemp(df):
    df['date'] = df['validTime'].map(dateFormat)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True).dt.date
    TMAX = df[['value','date']].groupby('date').max()
    TMAX = TMAX*1.8+32
    return TMAX.squeeze()
def minTemp(df):
    df['date'] = df['validTime'].map(dateFormat)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True).dt.date
    TMIN = df[['value','date']].groupby('date').min()
    TMIN = TMIN*1.8+32
    return TMIN.squeeze()
def avgWind(df):
    df['date'] = df['validTime'].map(dateFormat)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True).dt.date
    AWND = df.groupby('date').mean()
    AWND = AWND*0.621371
    return AWND.squeeze()
def precip(df):
    df['date'] = df['validTime'].map(dateFormat)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True).dt.date
    PRCP = df[['value','date']].groupby('date').sum()    
    PRCP = PRCP*0.0393701
    return PRCP.squeeze()

@retry(stop_max_attempt_number=10,wait_fixed=2000)
def getProperties(i,weather):
    Cities = ['Redding','Sacremento','Fresno','Bakersfield','Los Angeles']
    offices = ['STO','STO','HNX','HNX','LOX']
    gridpoints = ['27,163','40,67','51,100','67,35','154,44']
    DataCatagories = ['AWND','PRCP','TAVG','TMAX','TMIN']
    response = requests.get('https://api.weather.gov/gridpoints/{}/{}'.format(offices[i],gridpoints[i]))
    data = json.loads(response.text)
    weather['{} AWND'.format(Cities[i])] = avgWind(pd.DataFrame(data['properties']['windSpeed']['values']))
    weather['{} PRCP'.format(Cities[i])] = precip(pd.DataFrame(data['properties']['quantitativePrecipitation']['values']))
    weather['{} TAVG'.format(Cities[i])] = avgTemp(pd.DataFrame(data['properties']['temperature']['values']))
    weather['{} TMAX'.format(Cities[i])] = maxTemp(pd.DataFrame(data['properties']['temperature']['values']))
    weather['{} TMIN'.format(Cities[i])] = minTemp(pd.DataFrame(data['properties']['temperature']['values']))
    return weather


In [163]:
Cities = ['Redding','Sacremento','Fresno','Bakersfield','Los Angeles']
offices = ['STO','STO','HNX','HNX','LOX']
gridpoints = ['27,163','40,67','51,100','67,35','154,44']
DataCatagories = ['AWND','PRCP','TAVG','TMAX','TMIN']
weather = pd.DataFrame()
for i in range(0,len(Cities)):
    print(Cities[i])
    weather = getProperties(i,weather)

Redding
Sacremento
Fresno
Bakersfield
Los Angeles


In [164]:
weather

,Redding AWND,Redding PRCP,Redding TAVG,Redding TMAX,Redding TMIN,Sacremento AWND,Sacremento PRCP,Sacremento TAVG,Sacremento TMAX,Sacremento TMIN,...,Bakersfield AWND,Bakersfield PRCP,Bakersfield TAVG,Bakersfield TMAX,Bakersfield TMIN,Los Angeles AWND,Los Angeles PRCP,Los Angeles TAVG,Los Angeles TMAX,Los Angeles TMIN
date,,,,,,,,,,,,,,,,,,,,,
2020-08-01,4.794913,0.0,82.571429,102.0,68.0,7.364986,0.0,73.428571,96.0,59.0,...,4.730981,0.0,86.384615,103.0,72.0,5.063428,0.0,79.428571,93.0,69.0
2020-08-02,6.674519,0.0,87.521739,103.0,70.0,7.671861,0.0,77.173913,97.0,60.0,...,5.370302,0.0,87.476190,104.0,72.0,5.370302,0.0,75.842105,89.0,64.0
2020-08-03,7.480064,0.0,85.652174,102.0,70.0,6.674519,0.0,78.045455,97.0,62.0,...,5.610048,0.0,87.666667,102.0,73.0,8.515765,0.0,75.222222,86.0,65.0
2020-08-04,5.293584,0.0,85.500000,101.0,70.0,7.364986,0.0,74.727273,94.0,60.0,...,5.945692,0.0,85.333333,100.0,71.0,7.726660,0.0,73.388889,83.0,64.0
2020-08-05,5.466201,0.0,83.304348,100.0,69.0,8.745921,0.0,71.818182,89.0,59.0,...,5.753895,0.0,82.333333,97.0,69.0,5.984051,0.0,71.117647,80.0,62.0
2020-08-06,5.293584,0.0,82.173913,97.0,68.0,6.904675,0.0,72.857143,93.0,59.0,...,6.904675,0.0,79.857143,93.0,67.0,6.904675,0.0,68.705882,77.0,60.0
2020-08-07,4.833272,0.0,82.958333,98.0,68.0,7.767759,0.0,75.142857,92.0,60.0,...,5.753895,0.0,80.714286,95.0,68.0,6.214207,0.0,67.529412,78.0,58.0
2020-08-08,6.329285,NaN,83.791667,101.0,68.0,7.595142,NaN,74.772727,94.0,59.0,...,6.041590,0.0,81.952381,96.0,69.0,3.912649,0.0,68.470588,77.0,60.0
2020-08-09,9.206233,NaN,NaN,NaN,NaN,10.357012,NaN,NaN,NaN,NaN,...,5.753895,0.0,80.000000,94.0,70.0,NaN,0.0,NaN,NaN,NaN


In [165]:
X_predict = weather

In [166]:
import calendar
def MonthToString(month):
    return calendar.month_name[month]
X_predict.index = pd.to_datetime(X_predict.index)
X_predict['Day of Month'] = X_predict.index.get_level_values('date').day
X_predict['Month'] = X_predict.index.get_level_values('date').month
X_predict['Year'] = X_predict.index.get_level_values('date').year
X_predict['Day of Week'] = X_predict.index.get_level_values('date').weekday
DOW = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
X_predict['Day Of Week String'] = X_predict['Day of Week'].map(DOW)
X_predict['Month Name'] = X_predict['Month'].map(MonthToString)

In [167]:
X_predict['region'] = 'NP15'

In [168]:
capacity = pd.read_pickle('./Power_Plant_Capacities.pkl')

In [169]:
capacity

Hydro Capacity  Solar Capacity  Gas Capacity  Wind Capacity  \
Year Month                                                                
2001 1            15014.74           410.2      30378.07         1534.0   
     2            15014.74           410.2      30378.07         1534.0   
     3            15014.74           410.2      30378.07         1534.0   
     4            15014.74           410.2      30378.07         1534.0   
     5            15014.74           410.2      30378.07         1534.0   
...                    ...             ...           ...            ...   
2019 8            15565.39         13074.9      42296.51         6297.6   
     9            15565.39         13074.9      42296.51         6297.6   
     10           15565.39         13074.9      42296.51         6297.6   
     11           15565.39         13074.9      42296.51         6297.6   
     12           15565.39         13074.9      42296.51         6297.6   

            Geothermal Capacity  Nuclear Capacity  Coal Capacity  \
Year Month                                                         
2001 1                  2686.30            4456.0         4038.5   
     2                  2686.30            4456.0         4038.5   
     3                  2686.30            4456.0         4038.5   
     4                  2686.30            4456.0         4038.5   
     5                  2686.30            4456.0         4038.5   
...                         ...               ...            ...   
2019 8                  2785.55            2393.0         1898.3   
     9                  2785.55            2393.0         1898.3   
     10                 2785.55            2393.0         1898.3   
     11                 2785.55            2393.0         1898.3   
     12                 2785.55            2393.0         1898.3   

            Biomass Capacity  
Year Month                    
2001 1                887.70  
     2                887.70  
     3                887.70  
     4                887.70  
     5                887.70  
...                      ...  
2019 8               1087.36  
     9               1087.36  
     10              1087.36  
     11              1087.36  
     12              1087.36  

[228 rows x 8 columns]

In [170]:
X_predict['Hydro Capacity'] = capacity['Hydro Capacity'][2019,12]
X_predict['Solar Capacity'] = capacity['Solar Capacity'][2019,12]
X_predict['Gas Capacity'] = capacity['Gas Capacity'][2019,12]
X_predict['Wind Capacity'] = capacity['Wind Capacity'][2019,12]
X_predict['Geothermal Capacity'] = capacity['Geothermal Capacity'][2019,12]
X_predict['Nuclear Capacity'] = capacity['Nuclear Capacity'][2019,12]
X_predict['Coal Capacity'] = capacity['Coal Capacity'][2019,12]
X_predict['Biomass Capacity'] = capacity['Biomass Capacity'][2019,12]

In [171]:
X_predict = X_predict.drop(['Day of Month','Month','Year','Day of Week'],axis=1)

In [172]:
cols = ['region', 'Day Of Week String', 'Month Name', 'Redding AWND',
       'Redding PRCP', 'Redding TAVG', 'Redding TMAX', 'Redding TMIN',
       'Sacremento AWND', 'Sacremento PRCP', 'Sacremento TAVG',
       'Sacremento TMAX', 'Sacremento TMIN', 'Fresno AWND', 'Fresno PRCP',
       'Fresno TAVG', 'Fresno TMAX', 'Fresno TMIN', 'Bakersfield AWND',
       'Bakersfield PRCP', 'Bakersfield TAVG', 'Bakersfield TMAX',
       'Bakersfield TMIN', 'Los Angeles AWND', 'Los Angeles PRCP',
       'Los Angeles TAVG', 'Los Angeles TMAX', 'Los Angeles TMIN',
       'Hydro Capacity', 'Solar Capacity', 'Gas Capacity', 'Wind Capacity',
       'Geothermal Capacity', 'Nuclear Capacity', 'Coal Capacity',
       'Biomass Capacity']

In [173]:
X_predict = X_predict[cols]

In [174]:
import pickle
from joblib import dump, load

realTime = load('realtime_model.pkl')
dayAhead = load('dayahead_model.pkl')

In [175]:
realTime.predict(X_predict)

array([36.78998294, 36.78998823, 36.79010234, 36.78998085, 36.78998462,
       36.78999876, 36.79005084, 36.79004581, 36.79006193])

In [176]:
dayAhead.predict(X_predict)

array([38.10178274, 38.10183736, 38.10193627, 38.10178658, 38.10189751,
       38.10188534, 38.10181108, 38.10184904, 38.10204552])

In [177]:
X_predict

,region,Day Of Week String,Month Name,Redding AWND,Redding PRCP,Redding TAVG,Redding TMAX,Redding TMIN,Sacremento AWND,Sacremento PRCP,...,Los Angeles TMAX,Los Angeles TMIN,Hydro Capacity,Solar Capacity,Gas Capacity,Wind Capacity,Geothermal Capacity,Nuclear Capacity,Coal Capacity,Biomass Capacity
date,,,,,,,,,,,,,,,,,,,,,
2020-08-01,NP15,Saturday,August,4.794913,0.0,82.571429,102.0,68.0,7.364986,0.0,...,93.0,69.0,15565.39,13074.9,42296.51,6297.6,2785.55,2393.0,1898.3,1087.36
2020-08-02,NP15,Sunday,August,6.674519,0.0,87.521739,103.0,70.0,7.671861,0.0,...,89.0,64.0,15565.39,13074.9,42296.51,6297.6,2785.55,2393.0,1898.3,1087.36
2020-08-03,NP15,Monday,August,7.480064,0.0,85.652174,102.0,70.0,6.674519,0.0,...,86.0,65.0,15565.39,13074.9,42296.51,6297.6,2785.55,2393.0,1898.3,1087.36
2020-08-04,NP15,Tuesday,August,5.293584,0.0,85.500000,101.0,70.0,7.364986,0.0,...,83.0,64.0,15565.39,13074.9,42296.51,6297.6,2785.55,2393.0,1898.3,1087.36
2020-08-05,NP15,Wednesday,August,5.466201,0.0,83.304348,100.0,69.0,8.745921,0.0,...,80.0,62.0,15565.39,13074.9,42296.51,6297.6,2785.55,2393.0,1898.3,1087.36
2020-08-06,NP15,Thursday,August,5.293584,0.0,82.173913,97.0,68.0,6.904675,0.0,...,77.0,60.0,15565.39,13074.9,42296.51,6297.6,2785.55,2393.0,1898.3,1087.36
2020-08-07,NP15,Friday,August,4.833272,0.0,82.958333,98.0,68.0,7.767759,0.0,...,78.0,58.0,15565.39,13074.9,42296.51,6297.6,2785.55,2393.0,1898.3,1087.36
2020-08-08,NP15,Saturday,August,6.329285,NaN,83.791667,101.0,68.0,7.595142,NaN,...,77.0,60.0,15565.39,13074.9,42296.51,6297.6,2785.55,2393.0,1898.3,1087.36
2020-08-09,NP15,Sunday,August,9.206233,NaN,NaN,NaN,NaN,10.357012,NaN,...,NaN,NaN,15565.39,13074.9,42296.51,6297.6,2785.55,2393.0,1898.3,1087.36
